# Trenowanie modelu

In [1]:
import pandas as pd
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from scipy.sparse import csr_matrix
import numpy as np
import joblib 
from sklearn.preprocessing import LabelEncoder

### Reading dataset

In [ ]:
api=KaggleApi()
api.authenticate()

api.dataset_download_file(
    'paultimothymooney/recipenlg',
    file_name='RecipeNLG_dataset.csv',
    path='../data'
)

original_path = '../data/RecipeNLG_dataset.csv'
zip_path = '../data/RecipeNLG_dataset.zip'
os.rename(original_path, zip_path)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('../data')
os.remove(zip_path)

In [2]:
input_path='../data/RecipeNLG_dataset.csv'
print(f'Reading dataset from {input_path}')
data = pd.read_csv(input_path)
print(f'Dataset imported. Shape: {data.shape}')

Reading dataset from ../data/RecipeNLG_dataset.csv
Dataset imported. Shape: (2231142, 7)


### Czyszczenie zbioru

In [3]:
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
columns_to_drop = ["Unnamed: 0", "ingredients", "directions", "link", "source"]
data.drop(columns=columns_to_drop, inplace=True)
data.rename(columns={"NER": "ingredients"}, inplace=True)
print(f'Shape: {data.shape}')

Shape: (2231141, 2)


### Modyfikacja zbioru

In [4]:
data['ingredients'] = data['ingredients'].apply(ast.literal_eval)

mlb = MultiLabelBinarizer(sparse_output=True)
ingredient_matrix = mlb.fit_transform(data['ingredients'])

ingredient_df = pd.DataFrame.sparse.from_spmatrix(
    ingredient_matrix, 
    index=data.index, 
    columns=mlb.classes_
)

In [5]:
data = pd.concat([data, ingredient_df], axis=1)
data.drop(columns=['ingredients'], inplace=True)
data.drop(data.columns[2:15], axis=1, inplace=True)
print(f'Final shape: {data.shape}')

Final shape: (2231141, 234046)


### Trenowanie modelu

In [ ]:
# wybor najpopularniejszych przepisow
top_recipes = data['title'].value_counts().nlargest(300).index
data = data[data['title'].isin(top_recipes)]
print(f"Liczba unikalnych tytułów przepisów: {data['title'].nunique()}")

Liczba unikalnych tytułów przepisów: 100


In [9]:
# kodowanie title na liczby
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['title'])

# X - kolumny składników, y - zakodowane etykiety przepisow
ingredient_columns = data.columns.difference(['title', 'label'])
X = data[ingredient_columns]
y = data['label']

# podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# trenowanie prostego modelu
model = LogisticRegression(max_iter=1000, n_jobs=-1)
classifier = OneVsRestClassifier(model, n_jobs=-1)
classifier.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# zapis modelu
joblib.dump(classifier, 'recipe_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

### Ewaluacja

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Wyświetlanie składników danego przepisu (dla sprawdzenia poprawności testu)

In [ ]:
recipe = 'Chicken And Rice'
row = data[data['title'] == recipe].iloc[1]
ingredient_columns = data.columns.difference(['title'])
ingredients_used = row[ingredient_columns][row[ingredient_columns] == 1].index.tolist()

print(f"Składniki w {recipe}:")
print(ingredients_used)

[Debug] 100 Najczęściej występujących przepisów

In [ ]:
top = data['title'].value_counts().nlargest(100)
recipe in top.index